In [478]:
import numpy as np
import torch
import torch.nn as nn
import torch.utils.data as data
import snntorch as snn
import pandas as pd

In [479]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda')

In [480]:
# setting seeds
np.random.seed(445)
torch.manual_seed(445)

## Model

In [481]:
class SimpleSNNPredictor(nn.Module):
    def __init__(self, num_inputs, num_hidden, beta=0.95, num_steps=25):
        super().__init__()
        self.num_steps = num_steps

        # Initialize layers
        self.fc1 = nn.Linear(num_inputs, num_hidden)
        self.lif1 = snn.Leaky(beta=beta)
        self.fc2 = nn.Linear(num_hidden, num_outputs)

    def forward(self, x):

        # Initialize hidden states at t=0
        mem1 = self.lif1.init_leaky()

        for step in range(self.num_steps):
            cur1 = self.fc1(x)
            spk1, mem1 = self.lif1(cur1, mem1)
            cur2 = self.fc2(spk1)

        return cur2

In [482]:
def get_accuracy(model, loader, pct_close):
    correct = 0
    total = 0
    model.eval()
    for x, labels in loader:
        x, labels = x.to(device), labels.to(device)
        output = model(x)
        pred = torch.round(output)
        correct += (torch.abs(pred-labels.view_as(pred)) < torch.abs(pct_close * labels.view_as(pred))).sum().item()
        total += x.shape[0]
    return correct / total

In [483]:
batch_size = 32
dtype = torch.float

## Data

In [484]:
data06 = pd.read_csv("../data/processed/target06.csv")

In [485]:
all_data = data.TensorDataset(torch.from_numpy((data06.values[:,:-1] - data06.values[:,:-1].min(0)) / data06.values[:,:-1].ptp(0)).float(), torch.from_numpy(data06.values[:,-1]).float())  # with normalization
train_dataset, test_dataset, valid_dataset = torch.utils.data.random_split(all_data, (round(0.7 * len(all_data)), round(0.2 * len(all_data)), round(0.1 * len(all_data))))

In [486]:
train_loader = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, drop_last=False)
valid_loader = data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, drop_last=False)

## Prepare model

In [487]:
num_inputs=8
num_hidden=1200
num_outputs=1

model = SimpleSNNPredictor(num_inputs, num_hidden, num_outputs)
model.to(device)
print(model)

SimpleSNNPredictor(
  (fc1): Linear(in_features=8, out_features=1200, bias=True)
  (lif1): Leaky()
  (fc2): Linear(in_features=1200, out_features=1, bias=True)
)


In [488]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, betas=(0.9, 0.999))
loss_fun = nn.MSELoss()
num_steps = 20

## Training

In [489]:
num_epochs = 31
loss_hist = []
test_loss_hist = []

# Outer training loop
for epoch in range(num_epochs):
    for data, targets in train_loader:
        data = data.to(device)
        targets = targets.to(device)

    # forward pass
        model.train()
        preds = model(data).squeeze(dim=1)

        loss = loss_fun(preds, targets)

        # Gradient calculation + weight update
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Store loss history for future plotting
        #loss_hist.append(loss.item())

    if epoch % 2 == 0:
        print(f"Epoch: {epoch}, loss: {loss.item():.3}, acc: {get_accuracy(model, valid_loader, 0.2)}")

Epoch: 0, loss: 4.53e+02, acc: 0.3260115606936416
Epoch: 2, loss: 2.25e+02, acc: 0.3988439306358382
Epoch: 4, loss: 88.3, acc: 0.47398843930635837
Epoch: 6, loss: 50.6, acc: 0.49710982658959535
Epoch: 8, loss: 72.4, acc: 0.5132947976878612
Epoch: 10, loss: 46.4, acc: 0.522543352601156
Epoch: 12, loss: 70.1, acc: 0.5190751445086705
Epoch: 14, loss: 84.3, acc: 0.5179190751445086
Epoch: 16, loss: 1.2e+02, acc: 0.5063583815028901
Epoch: 18, loss: 80.1, acc: 0.5676300578034682
Epoch: 20, loss: 83.9, acc: 0.5398843930635838
Epoch: 22, loss: 48.5, acc: 0.5352601156069364
Epoch: 24, loss: 96.1, acc: 0.5190751445086705
Epoch: 26, loss: 58.5, acc: 0.5502890173410404
Epoch: 28, loss: 39.9, acc: 0.5421965317919075
Epoch: 30, loss: 77.1, acc: 0.5040462427745664


In [490]:
percent = 0.2
model.eval()
get_accuracy(model, test_loader, percent)

0.5624277456647399

In [491]:
targets

tensor([21.5200,  6.6400, 47.7700,  9.8000, 16.3400,  8.0200, 22.2200, 16.0000,
        26.5700, 14.7100, 14.2600, 46.8500, 12.0400, 13.7400, 10.8100,  7.8400,
        10.3400, 18.1300, 25.9300, 19.9300, 50.9400, 12.8100, 32.3200, 25.5500,
        43.8000,  7.4500, 21.7900, 30.9200, 18.9300, 34.4500, 70.2600, 29.7400],
       device='cuda:0')

In [492]:
preds

tensor([13.1635, 11.9892, 42.9729, 19.8671, 13.2309, 14.0652, 21.1927, 15.0116,
        29.9898, 14.6159, 15.4557, 35.8256, 24.2308,  7.5534, 18.2664,  8.2213,
        28.8393, 20.8042, 26.4876, 19.1598, 50.0451, 18.9561, 35.2028, 23.3641,
        32.4731, 24.9270, 30.6722, 28.0178, 37.1176, 51.6018, 60.0072, 16.8827],
       device='cuda:0', grad_fn=<SqueezeBackward1>)